# Notes

**Focus on**:
- Quality of data
- Log your data and calls when using data scraping
- Creativity
- Visualization (explanatory figures), simple is better
- Be critical of your data collection and generating process
    - Bias
    - Missing data
        - Ignore
        - Collect new data
        - Remove or replace missing data
    - Internal and external validity
    - Data collection type (random, survey, big data, other)
- Less focus on the analytical section and more on the collection and presentation

### Reflect on the ethical aspect
- Do yourespectprivacy? 
- Can single individualsbeidentified? 
- What are the potential consequences?
- Are there ethical considerations?
    - With respect to individuals? 
    - With respectto firmsor organizations?
- Consider the GDPR:
    - Is it anonymous? 
    - Personal data or statistics?
    - Any change of re-identification?

### Project ideas

- Factor based investing (use google or yahoo finance API)
- Trump tweet correlation with stock market
- Don't use LinkedIn
- 


### Logging

- Log your calls, use it to determine success ratio
    - Where did the call fail? Rewrite code.
    - Don't be greedy. time.sleep(0.5) between each call.
- Visualize the log (lecture 10)

In [5]:
# import twitter
# help(twitter)

Help on package twitter:

NAME
    twitter - A library that provides a Python interface to the Twitter API.

PACKAGE CONTENTS
    _file_cache
    api
    debug
    error
    models
    parse_tweet
    ratelimit
    twitter_utils

FUNCTIONS
    md5 = openssl_md5(...)
        Returns a md5 hash object; optionally initialized with a string

DATA
    __copyright__ = 'Copyright (c) 2007-2016 The Python-Twitter Developers...
    __description__ = 'A Python wrapper around the Twitter API'
    __download_url__ = 'https://pypi.python.org/pypi/python-twitter'
    __email__ = 'python-twitter@googlegroups.com'
    __license__ = 'Apache License 2.0'
    __url__ = 'https://github.com/bear/python-twitter'
    absolute_import = _Feature((2, 5, 0, 'alpha', 1), (3, 0, 0, 'alpha', 0...

VERSION
    3.5

AUTHOR
    The Python-Twitter Developers

FILE
    c:\users\bjorn\anaconda3\lib\site-packages\twitter\__init__.py




In [8]:
# Importing packages
import pandas as pd
import json, codecs

# Loading dataset from file using a relative path
with codecs.open('tweets.json', 'r', 'utf-8-sig') as json_file:  
    data = json.load(json_file)
    df=pd.DataFrame(data) # converting to a pandas data frame

display(df.head()) # Inspecting data frame

,created_at,favorite_count,id_str,is_retweet,retweet_count,source,text
0,Fri Jan 20 00:40:51 +0000 2017,165943,822242449053614081,False,34204,Twitter for iPhone,Thank you for joining us at the Lincoln Memori...
1,Fri Jan 20 04:24:33 +0000 2017,112829,822298747421986828,False,20770,Twitter for iPhone,Thank you for a wonderful evening in Washingto...
2,Fri Jan 20 12:31:53 +0000 2017,268372,822421390125043713,False,70523,Twitter for Android,It all begins today! I will see you at 11:00 A...
3,Fri Jan 20 17:51:25 +0000 2017,109640,822501803615014918,False,20125,Twitter for iPhone,Today we are not merely transferring power fro...
4,Fri Jan 20 17:51:58 +0000 2017,91143,822501939267141634,False,18362,Twitter for iPhone,"power from Washington, D.C. and giving it back..."
